PageRank Demo
=======================

This example runs PageRank on HiBench data using RAPIDS. Before running this notebook, you need to start a DASK scheduler and DASK workers in command line.

* dask-scheduler --scheduler-file path_to_scheduler_info_file_path # path_to_file_path: path to write the scheduler access information in the json format (e.g. ~\cluster.json to write to cluster.json under your home directory)

* mpirun -np number_of_workers --machinefile path_to_node_address_file dask-mpi --no-nanny --nthreads number_of_threads --no-scheduler --scheduler-file path_to_scheduler_info_file_path # numbrer_of_workers, path_to_node_address_file, and numbrer_of_threads need to be updated to proper values

1) Import Files
=======================


In [19]:
import os
import time
import numpy as np
from mpi4py import MPI
from dask.distributed import Client, wait
import dask
import dask_cudf
import dask_cugraph as dcg

2) Define Utility Functions
=======================

In [20]:
 def parse_host_port(address):
    if '://' in address:
        address = address.rsplit('://', 1)[1]
    host, port = address.split(':')
    port = int(port)
    return host, port

In [21]:
def set_visible(l, num_devs_per_node):
    try:
        comm = MPI.COMM_WORLD
        rank = comm.Get_rank()
        assert l == rank
        vd = l % num_devs_per_node
        os.environ["CUDA_VISIBLE_DEVICES"] = vd
        return vd
    
    except Exception as e:
        print(str(e))

3) Create a Client
=======================
Connect to the DASK scheduler, need to replace /home/seunghwak/cluster.json to the proper directory path.

In [22]:
start_time = time.time() # start timing from here

client = Client(scheduler_file = "/home/seunghwak/cluster.json", direct_to_workers = True) 
client

Client Scheduler: tcp://10.28.133.204:8786 Dashboard: http://10.28.133.204:8787/status,Cluster Workers: 2 Cores: 4 Memory: 16.42 GB


In [23]:
workers = [parse_host_port(worker) for worker in list(client.has_what().keys())]
workers

[('10.28.133.204', 38115), ('10.28.133.204', 46717)]

4) Map One Worker to One GPU
=======================

In [24]:
num_devs_per_node = 2;
assert (len(workers) % num_devs_per_node) == 0
pmpi_map = [(client.submit(set_visible, ident, num_devs_per_node, workers = [worker]), worker)
            for worker, ident in zip(workers, range(len(workers)))]
pmpi_map

[(<Future: status: pending, key: set_visible-2c8563b1fe87ea66e99c437f10933361>,
  ('10.28.133.204', 38115)),
 (<Future: status: pending, key: set_visible-554bc8900e51f118459f2ae225fc406a>,
  ('10.28.133.204', 46717))]

5) Read Input Data
=======================
Need to replace /home/seunghwak/TMP/Input-small/edges to the proper directory path

In [25]:
cdf = dask_cudf.read_csv("/home/seunghwak/TMP/Input-small/edges/part-*", delimiter='\t',  names=['src', 'dst'], 
                         dtype=['int32', 'int32'])
cdf = client.persist(cdf)
cdf

,src,dst
npartitions=32,,
,int32,int32
,...,...
...,...,...
,...,...
,...,...


In [26]:
cdf2 = cdf.sort_values_binned(by='dst') # data needs to be sorted based on destination index
cdf2 = client.persist(cdf2)
cdf2

,index,src,dst
npartitions=32,,,
,int64,int32,int32
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


6) Run PageRank
=======================

In [27]:
dcg.mg_pagerank(cdf2)

----------------> FUNTION NAME: MG_PAGERANK in dask_cugraph
----------------> FUNTION NAME: GET_MG_INFO
[<Future: status: error, key: ('from-delayed-9d4cbc0f89371fa4c24539043f397b71', 0)>, <Future: status: error, key: ('from-delayed-9d4cbc0f89371fa4c24539043f397b71', 1)>, <Future: status: error, key: ('from-delayed-9d4cbc0f89371fa4c24539043f397b71', 2)>, <Future: status: error, key: ('from-delayed-9d4cbc0f89371fa4c24539043f397b71', 3)>, <Future: status: error, key: ('from-delayed-9d4cbc0f89371fa4c24539043f397b71', 4)>, <Future: status: error, key: ('from-delayed-9d4cbc0f89371fa4c24539043f397b71', 5)>, <Future: status: error, key: ('from-delayed-9d4cbc0f89371fa4c24539043f397b71', 6)>, <Future: status: error, key: ('from-delayed-9d4cbc0f89371fa4c24539043f397b71', 7)>, <Future: status: error, key: ('from-delayed-9d4cbc0f89371fa4c24539043f397b71', 8)>, <Future: status: error, key: ('from-delayed-9d4cbc0f89371fa4c24539043f397b71', 9)>, <Future: status: error, key: ('from-delayed-9d4cbc0f893

StopIteration: 

7) Close the Client and Report Execution Time
=======================

In [36]:
client.close()

end_time = time.time()
print ((end_time - start_time), "seconds")

9.966898441314697 seconds
